## 找到回复review的例子，压缩后的数据有4个G，大约一亿行。

In [1]:
# 看一下数据的模样
import gzip
import pandas as pd
updates = pd.read_csv("/home/qiuyuanchen/data/UI_data/Reviews_Revisions.csv.gz", compression="gzip", iterator=True,chunksize=5)
next(updates)

,REVIEW_REVISION_HISTORY_ID,REVIEW_ID,APP_ID,PACKAGE_NAME,RATING,REVIEW_DATE,REVIEW_TITLE,REVIEW_TEXT,REPLY_TEXT,REPLY_DATE,REVIEWER_NAME,REVISION_TIMESTAMP
0,111320434,96003439,10669,AutomateIt.mainPackage,1,2015-08-09,Battery drain is bad bad!!,This app is draining my battery so bad I've ha...,NaN,NaN,Carl M,2018-10-14
1,111320435,96003440,10669,AutomateIt.mainPackage,5,2013-10-07,Sweet,Works like a charm.,NaN,NaN,Matt C,2018-10-14
2,111320436,96003441,10669,AutomateIt.mainPackage,1,2015-06-01,Broken,something changed now i cant set mobile data s...,NaN,NaN,Chris White,2018-10-14
3,111320437,96003442,10669,AutomateIt.mainPackage,5,2013-06-15,Excellent,Too bad ALL the features aren't free :-(,NaN,NaN,Mike Wickenden,2018-10-14
4,111320438,96003443,10669,AutomateIt.mainPackage,5,2015-06-27,NaN,Best app on this category!!,NaN,NaN,Avi N,2018-10-14


In [2]:
# 数据非常大，用iterator部分读取
update = pd.read_csv("/home/qiuyuanchen/data/UI_data/Reviews_Revisions.csv.gz", 
                    compression="gzip", 
                    usecols=["REVIEW_ID", "REVIEW_DATE", "REVIEW_TEXT", "REPLY_TEXT", "REPLY_TEXT"],
                    iterator=True,
                    chunksize=50000, )


In [3]:
# 获得数据的总数
num_lines = sum(1 for line in gzip.open('/home/qiuyuanchen/data/UI_data/Reviews_Revisions.csv.gz'))
print(num_lines)

101090214


In [4]:
from pprint import pprint
from IPython.display import display
from tqdm import tqdm
res = []
with tqdm(total=num_lines-1) as progress_bar:
    for chunck in update:
        progress_bar.update(len(chunck))
        chunck.dropna(subset=["REPLY_TEXT"], inplace=True)
        res.append(chunck)
replies = pd.concat(res)
print(replies.info())
replies.head()

100%|█████████▉| 101090213/101090214 [04:05<00:00, 412240.38it/s]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6892163 entries, 48 to 101090212
Data columns (total 4 columns):
REVIEW_ID      int64
REVIEW_DATE    object
REVIEW_TEXT    object
REPLY_TEXT     object
dtypes: int64(1), object(3)
memory usage: 262.9+ MB
None


,REVIEW_ID,REVIEW_DATE,REVIEW_TEXT,REPLY_TEXT
48,96003487,2015-06-03,If this worked more than 50 per cent of the ti...,Just released a new update that should fix sev...
126,96003565,2015-06-07,Tried many ways to turn on wifi none worked. ...,Just released a new update that should fix sev...
128,96003567,2014-03-10,All problems fixed :-),An update has been released with a fix for tha...
150,96003589,2013-07-11,Would like to be able to adjust GPS but that i...,The Enable/Disable GPS Action is supported for...
177,96003616,2014-02-11,Can't imagine my phone without it,Please refer to our FAQ page for a solution to...


In [9]:
groups = replies.groupby("REVIEW_ID")
print(len(groups))


6532385
96003487
    REVIEW_ID REVIEW_DATE                                        REVIEW_TEXT  \
48   96003487  2015-06-03  If this worked more than 50 per cent of the ti...   

                                           REPLY_TEXT  
48  Just released a new update that should fix sev...  
1
<class 'pandas.core.frame.DataFrame'>


In [84]:
dialogues = replies.drop_duplicates(subset=["REPLY_TEXT"], keep=False)
dialogues = dialogues[dialogues.duplicated(subset=["REVIEW_ID"], keep=False)]
groups = dialogues.groupby("REVIEW_ID")
print(len(groups))

17361


In [121]:
group_names = list(groups.groups.keys())
review_id = group_names[10115]
print(review_id)
sample = groups.get_group(review_id)
pd.set_option('display.max_colwidth', -1)
print(sample)

126867972
          REVIEW_ID REVIEW_DATE  \
31852058  126867972  2018-12-12   
32985529  126867972  2018-12-18   

                                                                                             REVIEW_TEXT  \
31852058  Wifi connected but says LAN is not connected. ( )                                                
32985529  Please enable shortcuts so we can navigate to a specific tool without multiple clicks. thanks!   

                                                                                                                                                                                                                              REPLY_TEXT  
31852058  Hi thanks for using this app and for your review. Just stay updated more tools utilities and improvements will be soon available. Please contact me at : info@eakteam.com to try to solve your issue. Thanks again              
32985529  Hi thanks for using this app and for your review. Just stay updated more tools 